to decode our .job format awe need to do the following in order

- parses metadata (dims, levels, wavelet, quant params)
- RLE-decodes coefficients
- dequantises (`0` treated as explicit zero)
- rebuilds the `wavedec2` coefficient tree using shapes inferred from metadata
- reconstructs via inverse DWT
- displays and saves `***_uncompressed.png`
- computes PSNR/SSIM if the original image is provided


In [ ]:
import os
import struct
import numpy as np
import pywt
import matplotlib.pyplot as plt
from PIL import Image

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

job_path = "TestPhotos/GREYtest_compressed.job"

#original file to see compression artifacts
original_image_path = "TestPhotos/GREYtest.png" 


Lets now extract the metadata from .job file to determine how to decode
structure is:
      uint16 width
      uint16 height
      uint8  L (levels)
      uint8  wavelet_name_length
      bytes  wavelet_name (ASCII)
      float32 threshold_fraction
      float32 min_val
      float32 step_size
      uint32 total_count          # expected = width * height
      uint32 encoded_count        # number of uint16 words following
      uint16[encoded_count] encoded_values (RLE stream)

In [ ]:
def read_job_header_and_payload(path: str):
    with open(path, "rb") as f:
        data = f.read()

    off = 0
    width  = struct.unpack_from("<H", data, off)[0]; off += 2
    height = struct.unpack_from("<H", data, off)[0]; off += 2
    L      = struct.unpack_from("<B", data, off)[0]; off += 1

    name_len = struct.unpack_from("<B", data, off)[0]; off += 1
    wavelet_name = data[off:off+name_len].decode("ascii"); off += name_len

    threshold_fraction = struct.unpack_from("<f", data, off)[0]; off += 4
    min_val = struct.unpack_from("<f", data, off)[0]; off += 4
    step_size = struct.unpack_from("<f", data, off)[0]; off += 4

    total_count = struct.unpack_from("<I", data, off)[0]; off += 4
    encoded_count = struct.unpack_from("<I", data, off)[0]; off += 4

    encoded = np.frombuffer(data, dtype="<u2", offset=off, count=encoded_count).copy()

    header = dict(
        width=width,
        height=height,
        L=L,
        wavelet_name=wavelet_name,
        threshold_fraction=threshold_fraction,
        min_val=min_val,
        step_size=step_size,
        total_count=total_count,
        encoded_count=encoded_count,
    )
    return header, encoded

header, encoded_values = read_job_header_and_payload(job_path)

print("Metadata:")
for k,v in header.items():
    print(f"  {k}: {v}")
print("Encoded stream uint16 length:", encoded_values.size)



We now need to reverse the entropy coding eg. re-adding the zeros that've been compresed

then compare with the origial provided length to check all is correct

In [ ]:
def rle_decode(encoded: np.ndarray, sentinel: int = 0xFFFF) -> np.ndarray:
    out = []
    i = 0
    n = int(encoded.size)
    while i < n:
        val = int(encoded[i]); i += 1
        if val != sentinel:
            out.append(val)
        else:
            if i >= n:
                raise ValueError("Malformed RLE stream: sentinel at end with no count.")
            run_len = int(encoded[i]); i += 1
            out.extend([0] * run_len)
    return np.array(out, dtype=np.uint16)

flat_q = rle_decode(encoded_values, sentinel=0xFFFF)
print("Decoded coefficient codes length:", flat_q.size, "(expected", header["total_count"], ")")
assert flat_q.size == header["total_count"], "Decoded length does not match header total_count."


rebuild coefficient tree, dequantise

    Wavedec2 coefficient structure 
      [cA_L, (cH_L,cV_L,cD_L), ..., (cH_1,cV_1,cD_1)]

    Dequantisation rule (matches encoder):
      - code == 0  -> coefficient = 0.0   (explicit zero from thresholding)
      - code > 0   -> coefficient ≈ code * step_size + min_val

In [ ]:
def infer_coeff_shapes(height: int, width: int, wavelet_name: str, L: int, mode: str = "symmetric"):
    '''
    Use a dummy zero image to infer the wavedec2 coefficient array shapes for (H,W,wavelet,L).
    Keeps the decoder flexible for wavelet choice and level count.
    '''
    dummy = np.zeros((height, width), dtype=np.float32)
    coeffs = pywt.wavedec2(dummy, wavelet_name, level=L, mode=mode)
    shapes = [coeffs[0].shape] + [tuple(arr.shape for arr in detail) for detail in coeffs[1:]]
    return shapes

def build_coeff_tree_from_flat(flat_q: np.ndarray, header: dict, mode: str = "symmetric"):

    #Reconstructs the wavedec2 coeff list:

    H = int(header["height"])
    W = int(header["width"])
    L = int(header["L"])
    wavelet = header["wavelet_name"]
    min_val = float(header["min_val"])
    step = float(header["step_size"])

    shapes = infer_coeff_shapes(H, W, wavelet, L, mode=mode)

    def dequantise(codes_1d: np.ndarray) -> np.ndarray:
        codes_f = codes_1d.astype(np.float32)
        out = codes_f * step + min_val
        out[codes_f == 0] = 0.0  # preserve explicit zeros
        return out

    idx = 0

    # Approximation cA_L
    cA_shape = shapes[0]
    nA = cA_shape[0] * cA_shape[1]
    cA_codes = flat_q[idx:idx+nA]; idx += nA
    cA = dequantise(cA_codes).reshape(cA_shape)

    coeffs = [cA]

    # tuples for each level: L down to 1
    for (sH, sV, sD) in shapes[1:]:
        nH = sH[0] * sH[1]
        nV = sV[0] * sV[1]
        nD = sD[0] * sD[1]

        cH = dequantise(flat_q[idx:idx+nH]).reshape(sH); idx += nH
        cV = dequantise(flat_q[idx:idx+nV]).reshape(sV); idx += nV
        cD = dequantise(flat_q[idx:idx+nD]).reshape(sD); idx += nD

        coeffs.append((cH, cV, cD))

    if idx != flat_q.size:
        raise ValueError(f"Did not consume the whole coefficient stream: used {idx}, total {flat_q.size}")

    return coeffs

coeffs_rec = build_coeff_tree_from_flat(flat_q, header, mode="symmetric")
print("Rebuilt coeff tree length:", len(coeffs_rec), "(should be L+1 =", header["L"]+1, ")")


we now run an inverse discrete wavelet transform (IDWT), and confirm we have the correct image size, then convert to uint8!!

In [ ]:
rec = pywt.waverec2(coeffs_rec, header["wavelet_name"], mode="symmetric")

# ensure correct dimensions (waverec2 can be wrong)
rec = rec[:header["height"], :header["width"]]

# Convert to uint8 for display/save
rec_u8 = np.clip(np.rint(rec), 0, 255).astype(np.uint8)

print("Reconstructed image shape:", rec_u8.shape, "dtype:", rec_u8.dtype)


we now check and save the image, if there already exists a file with our name, add a suffix

In [ ]:
plt.figure(figsize=(6, 6))
plt.imshow(rec_u8, cmap="gray", vmin=0, vmax=255)
plt.axis("off")
plt.title("Decoded image")
plt.show()

def name_available(path: str, sep: str = "_") -> str:
    folder, filename = os.path.split(path)
    stem, ext = os.path.splitext(filename)

    candidate = path
    i = 1
    while os.path.exists(candidate):
        candidate = os.path.join(folder, f"{stem}{sep}{i}{ext}")
        i += 1
    return candidate

out_path = root + "_decoded.png"
available_out_path = name_available(out_path)

root, _ = os.path.splitext(job_path.rsplit("_", 1)[0])

Image.fromarray(rec_u8, mode="L").save(available_out_path)
#Image.fromarray(rec_u8, mode="L").save(available_out_path, compress_level = 0) ideally want to do this so we only have our compression isntead of added lossless compression from png
print("Saved:", available_out_path)

#find compression ratio
input_size = os.path.getsize(original_image_path)
output_size = os.path.getsize(available_out_path)

print('compression ratio ',input_size / output_size)


we now calculate the errors created due to the lossy compression, we use PSNR and SSIM to see how the structure is changed



In [ ]:
if original_image_path is None:
    print("No original_image_path provided; skipping PSNR/SSIM.")
else:
    orig_u8 = np.array(Image.open(original_image_path).convert("L"), dtype=np.uint8)

    # Match shapes preemtivly
    H = min(orig_u8.shape[0], rec_u8.shape[0])
    W = min(orig_u8.shape[1], rec_u8.shape[1])
    orig_u8_c = orig_u8[:H, :W]
    rec_u8_c = rec_u8[:H, :W]

    print(f"PSNR: {psnr(orig_u8_c, rec_u8_c, data_range=255):.3f} dB")
    print(f"SSIM: {ssim(orig_u8_c, rec_u8_c, data_range=255):.6f}")


ideally we would a PSNR of above 30dB and a SSIM above 0.95 (higher is better for both). This could be achieved by using different wavelets, different quanatisation / 